In [169]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

### Fixed Values for Model

In [161]:
IA = 0 # deg
FZ = -500 # N

### Pacejka Fits

In [289]:
def lat_fit(data, a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17):
    FZ = data[0]
    SA = data[1]
    IA = data[2]

    C = a0
    D = FZ * (a1 * FZ + a2) * (1 - a15 * IA**2)
    
    BCD = a3 * np.sin(np.arctan(FZ / a4) * 2) * (1 - a5 * abs(IA))
    B = BCD / (C * D)
    H = a8 * FZ + a9 + a10 * IA

    E = (a6 * FZ + a7) * (1 - (a16 * IA + a17) * np.sign(SA + H))

    V = a11 * FZ + a12 + (a13 * FZ + a14) * IA * FZ
    Bx1 = B * (SA + H)

    return ((D * np.sin(C * np.arctan(Bx1 - E * (Bx1 - np.arctan(Bx1)))) + V), B, C)

def long_fit(data, b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13):
    FZ = data[0] / 1000 * -1
    SR = data[1]

    C = b0
    D = FZ * (b1 * FZ + b2)
    
    BCD = (b3 * FZ**2 + b4 * FZ) * np.exp(-1 * b5 * FZ)
    B = BCD / (C * D)
    H = b9 * FZ + b10

    E = (b6 * FZ**2 + b7 * FZ + b8) * (1 - b13 * np.sign(SR + H))

    V = b11 * FZ + b12
    Bx1 = B * (SR + H)

    return ((D * np.sin(C * np.arctan(Bx1 - E * (Bx1 - np.arctan(Bx1)))) + V), B, C)

### Comstock Equations

In [280]:
def com_lat(SA, SR, FX, FY, Cs):
    return ((FX * FY) / np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA * (np.pi / 180)))**2)) * (np.sqrt((1 - SR)**2 * (np.cos(SA * (np.pi / 180)))**2 * FY**2 + (np.sin(SA * (np.pi / 180)))**2 * Cs**2) / (Cs * np.cos(SA * (np.pi / 180))))
        
def com_long(SA, SR, FX, FY, Ca):
    return ((FX * FY) / np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA * (np.pi / 180)))**2)) * (np.sqrt(SR**2 * Ca**2 + (1 - SR)**2 * (np.cos(SA * (np.pi / 180)))**2 * FX**2) / Ca)

### Pacejka Fit Coefficients

In [281]:
front_lat = [0.349, -0.00115, 8.760, 730.300, 1745.322, 0.0139, -0.000277, 1.02025435, 0.000158, 0.149, -0.1595, 0.0329, 9.153,  0.00001406, 0.0328, 0.00362, -0.0143, -0.0116]

front_long = [0.771, 192.991, 3001.746, -403.002, 702.111, -0.492, 2.223, -1.97729069, -4.017, -2.669, 3.366, 2.044, -1.263, 0.353]

rear_lat = [1.384, -0.0003117, -2.936, 668.1, 1599, 0.03877, 0.0003177, 0.6252, 7.733e-05, -0.08382, -0.1171, 0.04597, 3.107, 5.41e-05, 0.04736, 0.005249, 0.0508, -0.1956]

rear_long = [0.9541497204155789, 192.8002686339246, 3003.620805462048, -399.72135093091333, 703.1031651893918, -0.6318690403434442, 1.4820764660971453, 0.9562715863473592, 
-3.2673329873816304, -0.7227790406912731, 0.7574230840392217, 0.8678660791229866, -4.582653049752284, -0.5261588380995105]

### Rear Tire

In [285]:
def comstock(SR, SA):
    FX_fit = long_fit([FZ, SR], *rear_long)
    FY_fit = lat_fit([FZ, SA, IA], *rear_lat)

    FX = FX_fit[0]
    FY = FY_fit[0]

    Ca = (long_fit([FZ, 1], *rear_long)[0] - long_fit([FZ, -1], *rear_long)[0]) / 2
    Cs = (lat_fit([FZ, 1, IA], *rear_lat)[0] - lat_fit([FZ, -1, IA], *rear_lat)[0]) / 2

    return (com_long(SA, SR, FX, FY, Ca), com_lat(SA, SR, FX, FY, Cs))

In [271]:
# print(comstock(0, 5)) # SR, SA
# print(long_fit([FZ, 0], *rear_long))

### Rear Tire Plots

In [290]:
model_SA_data = np.linspace(0.2, 25, 1000)
model_SR_data = np.linspace(0.2, 25, 1000)

SA, SR = np.meshgrid(model_SA_data, model_SR_data)

fig = plt.figure()
ax = Axes3D(fig, auto_add_to_figure=False)

FX = comstock(SR, SA)[0] * FZ * -1
FY = comstock(SR, SA)[1] * FZ * -1

ax = plt.axes(projection='3d')

fig.add_axes(ax)
ax.plot_surface(SR, SA, FY)

ax.set_xlabel('Slip Ratio (%)')
ax.set_ylabel('Slip Angle (deg)')
ax.set_zlabel('Longitudinal Force (kN)')

# plt.plot(FY, FX)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

<Figure size 640x480 with 0 Axes>